In [1]:
# load libraies

%run py_libraries.py

/Users/4476224/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.13.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# loading utility files

from utility.sv_fig import savefig
from utility.make_cm import make_confusion_matrix
from utility.get_g_result import get_gamma_results
from utility.plt_result import plot_results

In [3]:
def savefig(filename, crop = True):
    plt.savefig('{}.pdf'.format(filename))

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

# load data
X_train = pd.read_csv('data/norm_X_train_PCa_Ca.csv')
X_test = pd.read_csv('data/norm_X_test_PCa_Ca.csv')
y_train = pd.read_csv('data/y_train_PCa_Ca.csv')
y_test = pd.read_csv('data/y_test_PCa_Ca.csv')


print(X_test.shape)

(47, 37)


In [5]:
# Print the column names as a list
print(X_test.columns.tolist())

['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 'Leptin', 'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 'Laminin', 'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG']


In [6]:
Xy_PCa_Ca = pd.concat([X_test, y_test], axis=1)

In [7]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from scipy import stats

col_to_scale = ['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 'Leptin', 
                'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 'Laminin', 
                'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG' ]

def midas_impl(missing, col_to_scale, n_epoch):
    missing_cp = missing.copy() # make copy of missing df
    
    # scaler = MinMaxScaler().fit(missing_cp[col_to_scale]) # scale data
    scaler = StandardScaler().fit(missing_cp[col_to_scale]) # scale data
    missing_sc = scaler.transform(missing_cp[col_to_scale]) # this is an array, convert to df

    missing_df = pd.DataFrame({'ENA.78':missing_sc[:,0],
                               'IFN.y':missing_sc[:,1],
                               'IL.10':missing_sc[:,2],
                               'IL.6':missing_sc[:,3],
                               'IL.8':missing_sc[:,4],
                               'MCP.1':missing_sc[:,5],
                               'MDC':missing_sc[:,6],
                               'MIP.1a':missing_sc[:,7],
                               'TNF.a':missing_sc[:,8],
                               'C.peptide':missing_sc[:,9],
                               'G.CSF':missing_sc[:,10],
                               'IL.22':missing_sc[:,11],
                               'Insulin':missing_sc[:,12],
                               'Leptin':missing_sc[:,13],
                               'MIP.3a':missing_sc[:,14],
                               'GRO.a':missing_sc[:,15],
                               'HGF':missing_sc[:,16],
                               'MMP.2':missing_sc[:,17],
                               'Adiponectin':missing_sc[:,18],
                               'CRP':missing_sc[:,19],
                               'GDF.15':missing_sc[:,20],
                               'TIMP.1':missing_sc[:,21],
                               'TGF.B2':missing_sc[:,22],
                               'TGF.B1':missing_sc[:,23],
                               'PPAR.y':missing_sc[:,24],
                               'HIF.1a':missing_sc[:,25],
                               'Laminin':missing_sc[:,26],
                               'HbA1c':missing_sc[:,27],
                               'CA19.9':missing_sc[:,28],
                               'Glucose':missing_sc[:,29],
                               'HDL':missing_sc[:,30],
                               'CCK':missing_sc[:,31],
                               'LDL':missing_sc[:,32],
                               'Triglyceride':missing_sc[:,33],
                               'Albumin':missing_sc[:,34],
                               'Lumican':missing_sc[:,35],
                               'ZAG':missing_sc[:,36],
                               })
    
    # Extract cat columns from missing
    ext_col1 = missing_cp['CACHEXSTAGE0VIG']
    

    # Add the extracted cat columns to missing_df
    missing_df = pd.concat([missing_df, ext_col1.rename('CACHEXSTAGE0VIG')], axis=1)
    

    missing_df.to_csv('missing_test_PCa_Ca.csv')
    data_0 = pd.read_csv('missing_test_PCa_Ca.csv')
    data_0.columns.str.strip()

    categorical = ['CACHEXSTAGE0VIG']
    
    data_cat, cat_cols_list = md.cat_conv(data_0[categorical])

    data_0.drop(categorical, axis = 1, inplace = True)
    constructor_list = [data_0]
    constructor_list.append(data_cat)
    data_in = pd.concat(constructor_list, axis=1)

    na_loc = data_in.isnull()
    data_in[na_loc] = np.nan

    # imputer = md.Midas(layer_structure = [128,128], vae_layer = True, seed = 89, input_drop = 0.75, learn_rate = 0.0001)
    imputer = md.Midas(layer_structure = [128,128], vae_layer = False, seed = 42, input_drop = 0.75, learn_rate = 0.001)
    imputer.build_model(data_in, softmax_columns = cat_cols_list)
    imputer.train_model(training_epochs = n_epoch)

    # do not update M
    imputations = imputer.generate_samples(m=20).output_list

    flat_cats = [cat for variable in cat_cols_list for cat in variable]
    for i in range(len(imputations)):
        tmp_cat = [imputations[i][x].idxmax(axis=1) for x in cat_cols_list]
        cat_df = pd.DataFrame({categorical[i]:tmp_cat[i] for i in range(len(categorical))})
        imputations[i] = pd.concat([imputations[i], cat_df], axis = 1).drop(flat_cats, axis = 1)

    mapping_ca = {'CACHEXSTAGE0VIG_0.0': 0,'CACHEXSTAGE0VIG_1.0': 1}

    imputations[0] = imputations[0].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[1] = imputations[1].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[2] = imputations[2].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[3] = imputations[3].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[4] = imputations[4].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[5] = imputations[5].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[6] = imputations[6].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[7] = imputations[7].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[8] = imputations[8].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[9] = imputations[9].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[10] = imputations[10].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[11] = imputations[11].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[12] = imputations[12].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[13] = imputations[13].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[14] = imputations[14].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[15] = imputations[15].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[16] = imputations[16].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[17] = imputations[17].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[18] = imputations[18].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[19] = imputations[19].replace({'CACHEXSTAGE0VIG': mapping_ca})

    # rescale imputations
    imputations[0][col_to_scale] = scaler.inverse_transform(imputations[0][col_to_scale])
    imputations[1][col_to_scale] = scaler.inverse_transform(imputations[1][col_to_scale])
    imputations[2][col_to_scale] = scaler.inverse_transform(imputations[2][col_to_scale])
    imputations[3][col_to_scale] = scaler.inverse_transform(imputations[3][col_to_scale])
    imputations[4][col_to_scale] = scaler.inverse_transform(imputations[4][col_to_scale])
    imputations[5][col_to_scale] = scaler.inverse_transform(imputations[5][col_to_scale])
    imputations[6][col_to_scale] = scaler.inverse_transform(imputations[6][col_to_scale])
    imputations[7][col_to_scale] = scaler.inverse_transform(imputations[7][col_to_scale])
    imputations[8][col_to_scale] = scaler.inverse_transform(imputations[8][col_to_scale])
    imputations[9][col_to_scale] = scaler.inverse_transform(imputations[9][col_to_scale])
    imputations[10][col_to_scale] = scaler.inverse_transform(imputations[10][col_to_scale])
    imputations[11][col_to_scale] = scaler.inverse_transform(imputations[11][col_to_scale])
    imputations[12][col_to_scale] = scaler.inverse_transform(imputations[12][col_to_scale])
    imputations[13][col_to_scale] = scaler.inverse_transform(imputations[13][col_to_scale])
    imputations[14][col_to_scale] = scaler.inverse_transform(imputations[14][col_to_scale])
    imputations[15][col_to_scale] = scaler.inverse_transform(imputations[15][col_to_scale])
    imputations[16][col_to_scale] = scaler.inverse_transform(imputations[16][col_to_scale])
    imputations[17][col_to_scale] = scaler.inverse_transform(imputations[17][col_to_scale])
    imputations[18][col_to_scale] = scaler.inverse_transform(imputations[18][col_to_scale])
    imputations[19][col_to_scale] = scaler.inverse_transform(imputations[19][col_to_scale])

    n=0
    for i in imputations:
        file_out_csv = "midas_test_PCa_Ca/midas_PCa_Ca_" + str(n) + ".csv"
        i.to_csv(file_out_csv, index=False)
        n += 1

    df0 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_0.csv'); df0 = df0.iloc[:,1:]
    df1 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_1.csv'); df1 = df1.iloc[:,1:]
    df2 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_2.csv'); df2 = df2.iloc[:,1:]
    df3 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_3.csv'); df3 = df3.iloc[:,1:]
    df4 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_4.csv'); df4 = df4.iloc[:,1:]
    df5 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_5.csv'); df5 = df5.iloc[:,1:]
    df6 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_6.csv'); df6 = df6.iloc[:,1:]
    df7 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_7.csv'); df7 = df7.iloc[:,1:]
    df8 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_8.csv'); df8 = df8.iloc[:,1:]
    df9 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_9.csv'); df9 = df9.iloc[:,1:]
    df10 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_10.csv'); df10 = df10.iloc[:,1:]
    df11 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_11.csv'); df11 = df11.iloc[:,1:]
    df12 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_12.csv'); df12 = df12.iloc[:,1:]
    df13 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_13.csv'); df13 = df13.iloc[:,1:]
    df14 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_14.csv'); df14 = df14.iloc[:,1:]
    df15 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_15.csv'); df15 = df15.iloc[:,1:]
    df16 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_16.csv'); df16 = df16.iloc[:,1:]
    df17 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_17.csv'); df17 = df17.iloc[:,1:]
    df18 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_18.csv'); df18 = df18.iloc[:,1:]
    df19 = pd.read_csv('midas_test_PCa_Ca/midas_PCa_Ca_19.csv'); df19 = df19.iloc[:,1:]

    # Initialize the output dataframe with the same dimensions
    output_df = pd.DataFrame(index=df0.index, columns=df0.columns)

    # Process continuous columns
    for col in df1.columns[:35]:
        output_df[col] = (abs(df0[col]) + abs(df1[col]) + abs(df2[col]) + abs(df3[col]) + abs(df4[col]) + 
                          abs(df5[col]) + abs(df6[col]) + abs(df7[col]) + abs(df8[col]) + abs(df9[col]) + 
                         abs(df10[col]) + abs(df11[col]) + abs(df12[col]) + abs(df13[col]) + abs(df14[col]) + 
                          abs(df15[col]) + abs(df16[col]) + abs(df17[col]) + abs(df18[col]) + abs(df19[col])) / 20

    # Process categorical columns
    for col in df1.columns[35:]:
        output_df[col] = pd.concat([df0[col],df1[col],df2[col],df3[col],df4[col],df5[col],df6[col],df7[col],df8[col],df9[col],
                                   df10[col],df11[col],df12[col],df13[col],df14[col],df15[col],df16[col],df17[col],df18[col],df19[col]], axis=1).median(axis=1)
        
        # median = stats.median([df0[col],df1[col],df2[col],df3[col],df4[col],df5[col],df6[col],df7[col],df8[col],df9[col],
        #                   df10[col],df11[col],df12[col],df13[col],df14[col],df15[col],df16[col],df17[col],df18[col],df19[col]], axis=0).median[0]
        # output_df[col] = median

    # Write the output dataframe to a CSV file
    output_df.to_csv('output_test_PCa_Ca.csv', index=False)

    # re-order cols in output_df
    output_df_re = output_df[['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 'Leptin', 
                              'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 'Laminin', 'HbA1c', 
                              'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG', 'CACHEXSTAGE0VIG']].copy()

    return output_df_re





In [8]:
# calculate proportion of missingness in ndata_noR

# total num of NaN in the ndata_noR
total_nan_count = X_test.isna().sum().sum()

# total num of cells in ndata_noR
total_cells = X_test.size

# proportion of NaN
NaN_proportion = total_nan_count / total_cells

print('NaN_proportions:',NaN_proportion)


NaN_proportions: 0.01437607820586544


In [9]:
# 1.35% missingness

Xy_PCa_Ca_md = midas_impl(Xy_PCa_Ca, col_to_scale, 200)

Size index: [38, 2]

Computation graph constructed

Metal device set to: Apple M1 Pro
Model initialised

Epoch: 0 , loss: 82.92953109741211
Epoch: 1 , loss: 77.43391036987305
Epoch: 2 , loss: 76.8089599609375
Epoch: 3 , loss: 80.95058822631836
Epoch: 4 , loss: 87.54781341552734
Epoch: 5 , loss: 83.17686080932617
Epoch: 6 , loss: 109.87956619262695
Epoch: 7 , loss: 77.93888473510742
Epoch: 8 , loss: 81.28152465820312
Epoch: 9 , loss: 78.32391357421875
Epoch: 10 , loss: 77.26318359375
Epoch: 11 , loss: 67.4437370300293
Epoch: 12 , loss: 73.42452621459961
Epoch: 13 , loss: 72.33415603637695
Epoch: 14 , loss: 59.05307197570801
Epoch: 15 , loss: 72.62359619140625
Epoch: 16 , loss: 80.94826126098633
Epoch: 17 , loss: 98.83183670043945
Epoch: 18 , loss: 79.33412551879883
Epoch: 19 , loss: 82.16909408569336
Epoch: 20 , loss: 69.56526184082031
Epoch: 21 , loss: 77.05895233154297
Epoch: 22 , loss: 77.11618423461914
Epoch: 23 , loss: 89.66184616088867
Epoch: 24 , loss: 109.9769515991211
Epoch: 25

In [10]:
Xy_PCa_Ca_md

,ENA.78,IFN.y,IL.10,IL.6,IL.8,MCP.1,MDC,MIP.1a,TNF.a,C.peptide,G.CSF,IL.22,Insulin,Leptin,MIP.3a,GRO.a,HGF,MMP.2,Adiponectin,CRP,GDF.15,TIMP.1,TGF.B2,TGF.B1,PPAR.y,HIF.1a,Laminin,HbA1c,CA19.9,Glucose,HDL,CCK,LDL,Triglyceride,Albumin,Lumican,ZAG,CACHEXSTAGE0VIG
0,0.985370,0.653584,0.505136,0.406178,0.722950,0.822711,0.839128,0.542819,0.755327,0.870083,0.848500,0.323372,0.774950,0.723358,0.408217,0.971083,0.616233,0.912034,0.889173,0.726518,0.761615,0.895058,0.958150,0.979735,0.536159,0.668981,0.812854,0.793340,0.507235,0.771969,0.943904,0.806855,0.820744,0.640146,0.985239,0.971775,0.957044,0.0
1,0.765643,0.584304,0.366265,0.514657,0.722746,0.857897,0.865491,0.555477,0.723024,0.911624,0.581307,0.181021,0.748840,0.902810,0.480132,0.800747,0.648594,0.916385,0.931320,0.844876,0.823089,0.892452,0.840748,0.860759,0.285906,0.747851,0.904832,0.822087,0.308300,0.742466,0.956585,0.852123,0.886628,0.697797,0.990153,0.949113,0.963158,1.0
2,0.838268,0.921745,0.519779,0.500294,0.821011,1.040662,0.870819,0.553589,0.672355,0.904048,0.653972,0.394620,0.930521,0.725714,0.551996,0.797949,0.641887,0.926116,0.919166,0.817089,0.819160,0.897393,0.753235,0.943235,0.447061,0.429549,0.943499,0.768935,0.632631,0.778828,0.798110,0.897758,0.837388,0.761565,0.984791,0.905909,0.917273,1.0
3,0.806237,0.308339,0.544211,0.673155,1.000646,0.871355,0.854521,0.597727,0.637839,0.862640,0.474086,0.007128,0.604335,0.535287,0.685997,0.888787,0.800122,0.859641,0.938919,0.993226,1.106045,1.005314,0.866417,0.925750,0.636899,0.851536,1.001409,0.891588,0.348931,0.911832,0.800792,0.922825,0.833575,0.868490,0.980347,0.909700,0.921061,0.0
4,0.818214,0.595559,0.119467,0.386944,0.689603,0.817603,0.840238,0.506400,0.601727,0.891299,0.682757,0.145394,0.858153,0.867506,0.460349,0.747186,0.662913,0.857597,0.921624,0.827844,0.825388,0.894088,0.665592,0.915234,0.396857,0.793249,0.880276,0.805432,0.704015,0.825421,0.827834,0.769575,0.963486,0.664006,0.970991,0.925938,0.939352,0.0
5,0.986491,0.549236,0.472155,0.487651,0.868769,0.838446,0.854731,0.592034,0.771497,0.804515,0.603814,0.237356,0.570404,0.314521,0.536737,0.965004,0.645251,0.723378,0.884034,0.888228,0.868975,0.891704,0.788018,0.946776,0.362058,0.737757,0.971906,0.822890,0.860879,0.817136,0.839917,0.850854,0.810526,0.749057,0.976524,0.888719,0.915452,0.0
6,0.821819,0.655201,0.588366,0.524530,0.687277,0.851870,0.874605,0.584725,0.784458,0.821397,0.614404,0.235646,0.555700,0.798071,0.383927,0.760095,0.612580,0.958025,0.949911,0.850176,0.911106,0.873260,0.756540,0.899229,0.435101,0.956715,0.892491,0.745712,0.461641,0.972956,0.878639,0.630239,0.923082,0.776338,0.975591,0.971275,0.964977,0.0
7,0.776261,0.722241,0.444740,0.513168,0.692927,0.870929,0.859415,0.590320,0.749498,0.863446,0.921386,0.053873,0.633316,0.773515,0.451839,0.792087,0.669136,0.949166,0.926196,0.711908,0.814770,0.859554,0.416464,0.895157,0.860208,0.688284,0.842874,0.768693,0.515806,0.796872,0.860907,0.794252,0.836577,0.595763,0.989060,0.928173,0.961682,0.0
8,0.935920,0.523604,0.204800,0.414409,0.724465,0.930432,0.918320,0.599057,0.695536,0.930102,0.640579,0.148396,0.827867,0.872385,0.533158,0.846314,0.667063,0.931233,0.945386,0.795654,0.847920,0.905633,0.698395,0.964025,0.814240,0.738557,0.869140,0.825358,0.349376,0.742542,0.917268,0.809144,0.821643,0.501511,0.980347,0.904151,0.945306,0.0
9,0.782240,0.596993,0.414491,0.437085,0.672095,0.842620,0.816450,0.614558,0.673856,0.902095,0.612587,0.237050,0.829065,0.721566,0.547053,0.730497,0.619959,0.957000,0.960402,0.599841,0.909392,0.868221,0.529731,0.915287,0.409220,0.714189,0.817147,0.824207,0.373990,0.828845,0.821550,0.728978,0.789249,0.737953,0.985703,0.980426,0.992655,0.0


In [11]:
# save to csv file

# 
Xy_PCa_Ca_md.to_csv('data/Xy_test_PCa_Ca_md.csv', index=False)